# Part II: Model Development

In this part, we develop three unique pipelines for predicting backorder. We use the smart sample from Part I to fit and evaluate these pipelines. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd
import joblib

from sklearn import preprocessing
from sklearn.model_selection import train_test_split


## Reload the smart sample here

In [2]:
# Reload your smart sampling from local file 
# ----------------------------------
X, y, train_undersamp = joblib.load('data/sample-data-v4.pkl')


In [3]:
# Subset easier to manage size for testing pipelines
train_undersamp_less = train_undersamp.groupby('went_on_backorder').apply(lambda x: x.sample(frac=0.1))
train_undersamp_less = pd.DataFrame(train_undersamp_less)
# Split back into X and y
X_less = train_undersamp_less.iloc[:, :-1]
y_less = train_undersamp_less.went_on_backorder
X_less.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2258 entries, (0, 7480) to (1, 18291)
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   national_inv      2258 non-null   float64
 1   lead_time         2258 non-null   float64
 2   in_transit_qty    2258 non-null   float64
 3   forecast_3_month  2258 non-null   float64
 4   sales_1_month     2258 non-null   float64
 5   sales_3_month     2258 non-null   float64
 6   min_bank          2258 non-null   float64
 7   potential_issue   2258 non-null   int64  
 8   pieces_past_due   2258 non-null   float64
 9   perf_6_month_avg  2258 non-null   float64
 10  local_bo_qty      2258 non-null   float64
 11  deck_risk         2258 non-null   int64  
 12  oe_constraint     2258 non-null   int64  
 13  ppap_risk         2258 non-null   int64  
 14  stop_auto_buy     2258 non-null   int64  
 15  rev_stop          2258 non-null   int64  
dtypes: float64(10), int64(6)
mem

## Normalize/standardize the data if required; otherwise ignore. You can perform this step inside the pipeline (if required). 

In [4]:
# Standardize data
scaler = preprocessing.StandardScaler().fit(X_less)
X_less_scaled = scaler.transform(X_less)

'''# Combine X_scaled and y
train_under_stand = pd.DataFrame()
train_under_stand = X_scaled
train_under_stand.reset_index(drop = True)
train_under_stand['went_on_backorder'] = y'''

print("Mean of the dataset features")
print(scaler.mean_)
print("Variance of data")
print(scaler.scale_)
print("-" * 35)

print("# Scaled data:")
print(X_less_scaled)

print("# Mean of scaled data")
print(X_less_scaled.mean(axis = 0))
print("# Variance of scaled data")
print(X_less_scaled.std(axis = 0))


Mean of the dataset features
[3.13459256e+02 7.38777150e+00 2.83064659e+01 1.80642161e+02
 4.16133747e+01 1.30127104e+02 4.16767050e+01 6.20017715e-03
 1.10008857e+01 3.22648621e-01 2.93755536e+00 2.05048716e-01
 0.00000000e+00 1.54118689e-01 9.53498671e-01 0.00000000e+00]
Variance of data
[4.34992106e+03 6.54284649e+00 7.90264266e+02 2.72756146e+03
 7.45954297e+02 2.50537037e+03 7.07191300e+02 7.84967194e-02
 4.36635829e+02 1.76743406e+00 3.42636588e+01 4.03737216e-01
 1.00000000e+00 3.61062486e-01 2.10568172e-01 1.00000000e+00]
-----------------------------------
# Scaled data:
[[-0.05987678  0.09357219 -0.03581899 ... -0.42684769  0.2208374
   0.        ]
 [ 0.05713684 -0.51778251 -0.03581899 ... -0.42684769  0.2208374
   0.        ]
 [-0.06746312  0.09357219 -0.03581899 ... -0.42684769  0.2208374
   0.        ]
 ...
 [-0.07160113  0.09357219 -0.03581899 ... -0.42684769  0.2208374
   0.        ]
 [ 0.08886155  0.7049269   0.84996066 ... -0.42684769  0.2208374
   0.        ]
 [-0.072

In [5]:
''' # Subset non discrete features
X_less_for_norm = X_less[['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
                          'sales_1_month', 'sales_3_month', 'min_bank', 'pieces_past_due', 
                          'perf_6_month_avg', 'local_bo_qty']]
#X_less_for_norm.info()

# Subset discrete features
X_less_discrete = pd.DataFrame(X_less[['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 
                                       'stop_auto_buy', 'rev_stop']])
#X_less_discrete.info()

# Normalize data
X_less_norm = pd.DataFrame(preprocessing.normalize(X_less_for_norm, axis = 0, norm = 'l2'))
#X_less_norm = pd.DataFrame(X_less_norm)
#X_less_norm.info()

#print('# Scaled & normalized values:')
#print(X_less_norm)

#print("# All have unit norm")
#print(np.linalg.norm(X_less_norm, axis = 0))

#X_less_norm.info()

# Recombine discrete and nondiscrete features
X_less_norm_comb = pd.DataFrame(pd.concat([X_less_norm, X_less_discrete]).reset_index(drop = True))

X_norm = X_less_norm_comb.rename(columns={0: 'national_inv', 1: 'lead_time', 2: 'in_transit_qty', 
                                 3: 'forecast_3_month', 4: 'sales_1_month', 5: 'sales_3_month', 
                                 6: 'min_bank' , 7: 'potential_issue', 8: 'pieces_past_due', 
                                 9: 'perf_6_month_avg', 10: 'local_bo_qty', 11: 'deck_risk', 
                                 12: 'oe_constraint', 13: 'ppap_risk', 14: 'stop_auto_buy', 
                                 15: 'rev_stop'})
#X_less_norm_comb.info()
#column = X.columns.values
#print(column)
#X_norm.info()

#check = pd.DataFrame(X_norm['potential_issue'].unique())
#print(check)

X_norm = pd.Series(X_less_norm_comb)

print('potential_issue', X_norm['potential_issue'].unique())
print('/deck_risk', X_norm['deck_risk'].unique())
print('/oe_constraint', X_norm['oe_constraint'].unique())
print('/ppap_risk', X_norm['ppap_risk'].unique())
print('/stop_auto_buy', X_norm['stop_auto_buy'].unique())
print('/rev_stop', X_norm['rev_stop'].unique())
print('/went_on_backorder', X_norm['went_on_backorder'].unique())

potential_issue

for col in X_norm:
  print(X_norm[col].unique()) '''


' # Subset non discrete features\nX_less_for_norm = X_less[[\'national_inv\', \'lead_time\', \'in_transit_qty\', \'forecast_3_month\',\n                          \'sales_1_month\', \'sales_3_month\', \'min_bank\', \'pieces_past_due\', \n                          \'perf_6_month_avg\', \'local_bo_qty\']]\n#X_less_for_norm.info()\n\n# Subset discrete features\nX_less_discrete = pd.DataFrame(X_less[[\'potential_issue\', \'deck_risk\', \'oe_constraint\', \'ppap_risk\', \n                                       \'stop_auto_buy\', \'rev_stop\']])\n#X_less_discrete.info()\n\n# Normalize data\nX_less_norm = pd.DataFrame(preprocessing.normalize(X_less_for_norm, axis = 0, norm = \'l2\'))\n#X_less_norm = pd.DataFrame(X_less_norm)\n#X_less_norm.info()\n\n#print(\'# Scaled & normalized values:\')\n#print(X_less_norm)\n\n#print("# All have unit norm")\n#print(np.linalg.norm(X_less_norm, axis = 0))\n\n#X_less_norm.info()\n\n# Recombine discrete and nondiscrete features\nX_less_norm_comb = pd.DataFrame(

## Split the data into Train/Test

In [6]:
## Split downsampled dataset into training and testing sets
# Small sample to test pipelines
X_train, X_test, y_train, y_test = train_test_split(X_less, y_less, test_size = 0.2)
# Full sample
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


## Developing Pipeline

In this section, we design an operationalized machine learning pipeline, which includes:

* Anomaly detection
* Dimensionality Reduction
* Train a classification model


We are free to use any of the models that we learned in the past or we can use new models. Here is a pool of methods: 

### Pool of Anomaly Detection Methods (Discussed in M4)
1. IsolationForest
2. EllipticEnvelope
3. LocalOutlierFactor
4. OneClassSVM
5. SGDOneClassSVM

### Pool of Feature Selection Methods (Discussed in M3)

1. VarianceThreshold
1. SelectKBest with any scoring method (e.g, chi, f_classif, mutual_info_classif)
1. SelectKPercentile
3. SelectFpr, SelectFdr, or  SelectFwe
1. GenericUnivariateSelect
2. PCA
3. Factor Analysis
4. Variance Threshold
5. RFE
7. SelectFromModel


### Classification Methods (Discussed in M1-M2
1. Decision Tree
2. Random Forest
3. Logistic Regression
4. Naive Bayes
5. Linear SVC
6. SVC with kernels
7. KNeighborsClassifier
8. GradientBoostingClassifier
9. XGBClassifier
10. LGBM Classifier



It is difficult to fit an anomaly detection method in the sklearn pipeline without writing custom codes. For simplicity, we avoid fitting an anomaly detection method within a pipeline. So we can create the workflow in two steps. 
* Step I: fit an outlier with the training set
* Step II: define a pipeline using a feature selection and a classification method. Then cross-validate this pipeline using the training data without outliers. 
* Note: if your smart sample is somewhat imbalanced, you might want to change the scoring method in GridSearchCV (see the [doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).


Once we fit the pipeline with gridsearch, we identify the best model and give an unbiased evaluation using the test set that we created in Part II. For unbiased evaluation we report confusion matrix, precision, recall, f1-score, accuracy, and other measures if you like. 

**Optional: Those who are interested in writing custom codes for adding an outlier detection method into the sklearn pipeline, please follow this discussion [thread](https://stackoverflow.com/questions/52346725/can-i-add-outlier-detection-and-removal-to-scikit-learn-pipeline).**


**Note:** <span style='background:yellow'>We will be using Grid Search to find the optimal parameters of the pipelines.</span>

You can add more notebook cells or import any Python modules as needed.

In [7]:
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.feature_selection import (SelectKBest, VarianceThreshold, 
                                       SelectPercentile, chi2, f_classif, mutual_info_classif)
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix 
from numpy import where
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline

import sklearn.feature_selection


### Your 1st pipeline 
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation
  
Add cells as needed. 

In [8]:
# Add anomaly detection code  (Question #E201)
# ----------------------------------
# Construct local outlier factor
lof = LocalOutlierFactor()

# Get labels from classifier to cull outliers
lof_outliers = lof.fit_predict(X_train) == -1 # This should be scaled for LR, but I could not make it work

X_lof = X_train[~lof_outliers]
y_lof = y_train[~lof_outliers]


In [9]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E202)
# ----------------------------------
# Define pipeline
pipe = Pipeline([
    ('selector', PCA()),
    ('classifier', LogisticRegression())
])

# Define parameter grid
param_grid = {'selector__n_components': [0, 1, 2, 3],
              'selector__random_state':[17],
              'classifier__C': [0.001, 0.1, 1.0, 10, 100], 
              'classifier__max_iter': [1000, 2500, 5000]
              }

# Use Grid Search to train Pipeline
CV_log_reg = GridSearchCV(pipe, param_grid, n_jobs = 2, cv = 10)
CV_log_reg_model = CV_log_reg.fit(X_lof, y_lof)
#print(CV_log_reg.cv_results_)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.50842061 0.75677838 0.76195224        nan 0.50842061
 0.75677838 0.76195224        nan 0.50842061 0.75677838 0.76195224
        nan 0.50842061 0.76000838 0.76453289        nan 0.50842061
 0.76000838 0.76453289        nan 0.50842061 0.76000838 0.76453289
        nan 0.50842061 0.76000838 0.76453289        nan 0.50842061
 0.76000838 0.76453289        nan 0.50842061 0.76000838 0.76453289
        nan 0.50842061 0.76000838 0.76453289        nan 0.50842061
 0.76000838 0.76453289        nan 0.50842061 0.76000838 0.76453289
        nan 0.50842061 0.76000838 0.76453289        nan 0.50842061
 0.76000838 0.76453289        nan 0.50842061 0.76000838 0.76453289]
  category=UserWarning


In [10]:
# Given an unbiased evaluation  (Question #E203)
# ----------------------------------
'''# Show parameters of trained models and their rank
pd.set_option("max_colwidth", 80)
CV_log_reg_df = pd.DataFrame(CV_log_reg.cv_results_)
print(CV_log_reg_df[['params','rank_test_score']])'''

# Evaluate best model using test data
predicted_y = CV_log_reg.predict(X_test)

'''# Show parameters of best model
best_params = CV_log_reg.best_params_
print('Best parameter:\n',best_params)'''

# Show best estimator
best_estimator = CV_log_reg.best_estimator_
print('Best estimator:\n',best_estimator) 

# Display confusion matrix
print('\nConfusion Matrix:\n',pd.DataFrame(confusion_matrix(y_test, predicted_y)))

# Create classification report
print('\nClassification Report:\n',classification_report(y_test, predicted_y))


Best estimator:
 Pipeline(steps=[('PCA', PCA(n_components=3, random_state=17)),
                ('LR_model', LogisticRegression(C=0.1, max_iter=1000))])

Confusion Matrix:
      0    1
0  145   87
1   22  181

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.62      0.73       232
           1       0.68      0.89      0.77       203

    accuracy                           0.75       435
   macro avg       0.77      0.76      0.75       435
weighted avg       0.78      0.75      0.75       435



#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 2nd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [8]:
# Add anomaly detection code  (Question #E205)
# ----------------------------------
# Construct envelope
ee = EllipticEnvelope()
# Fit data to envelope
#envelope = env.fit(X_train_norm, y_train_norm)
envelope = ee.fit(X_train, y_train)

# Get labels from classifier to cull outliers
env_outliers = envelope.predict(X_train) == -1

# Re-slice X,y into a cleaned dataset with outliers excluded
X_env = X_train[~env_outliers]
y_env = y_train[~env_outliers]


/opt/conda/lib/python3.7/site-packages/sklearn/covariance/_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


In [16]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E206)
# ----------------------------------
#kb = SelectKBest()
#rfc = RandomForestClassifier(random_state = 17)
#fa = FactorAnalysis()
#dtc = DecisionTreeClassifier()

# Define pipeline
'''pipe2 = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('classifier', rfc)
        ('selector', FactorAnalysis()),
    ]
)

param_grid2 = [
    {
        'selector__n_components': [5, 20, 80, 120, 480],
        'classifier__max_features': ['auto', 'sqrt', 'log2'],
        'classifier__max_depth': [5, 15, 10, 5, 1],
        'classifier__random_state': [17]
    },
]'''

pipe2 = Pipeline(
    [
        ('selector', SelectKBest(mutual_info_classif)),
        ('classifier', DecisionTreeClassifier())
    ]
)

param_grid2 = [
    {
        'selector__k': [1, 3, 5, 10, 15],
        'classifier__max_features': ['auto', 'sqrt', 'log2'],
        'classifier__max_leaf_nodes': [None, 15, 10, 5, 1],
        'classifier__random_state': [17]
    },
]

# Use Grid Search to train Pipeline
CV_dtc = GridSearchCV(pipe2, param_grid = param_grid2, n_jobs = 2, cv = 10)
CV_dtc_model = CV_dtc.fit(X_env, y_env)
#print(CV_dtc.cv_results_)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.76984019 0.82153677 0.82029842 0.81105052 0.80797925 0.77163145
 0.79079755 0.84057032 0.82091949 0.8214989  0.79383852 0.77479739
 0.82457775 0.79757252 0.83076574 0.7950693  0.72308188 0.81474665
 0.78893812 0.80794895        nan        nan        nan        nan
        nan 0.76984019 0.81784822 0.81659471 0.81044081 0.80429827
 0.7889154  0.77965614 0.82950845 0.80613497 0.82213512 0.76729531
 0.77109369 0.83133757 0.80740741 0.83199652 0.7950693  0.7378929
 0.81658335 0.78954026 0.81659093        nan        nan        nan
        nan        nan 0.75440809 0.82278649 0.80916458 0.81287965
 0.80554419 0.78523442 0.78339014 0.84058169 0.80678255 0.84308112
 0.78340907 0.77479739 0.83443536 0.81721578 0.83570779 0.7950693
 0.72555101 0.81780656 0.77717943 0.81722336        nan        nan
        nan        nan        nan]
  category=UserWarning


In [17]:
# Given an unbiased evaluation  (Question #E207)
# ----------------------------------
# Evaluate best model using test data
predicted_y = CV_dtc.predict(X_test)

'''# Show parameters of best model
best_params = CV_dtc.best_params_
print('Best parameter:\n',best_params)'''

# Show best estimator
best_estimator = CV_dtc.best_estimator_
print('Best estimator:\n',best_estimator) 

# Display confusion matrix
print('\nConfusion Matrix:\n',pd.DataFrame(confusion_matrix(y_test, predicted_y)))

# Create classification report
print('\nClassification Report:\n',classification_report(y_test, predicted_y))


Best estimator:
 Pipeline(steps=[('kbest',
                 SelectKBest(k=15,
                             score_func=<function mutual_info_classif at 0x7fb4e0f1d8c8>)),
                ('dtc_model',
                 DecisionTreeClassifier(max_features='log2', max_leaf_nodes=15,
                                        random_state=17))])

Confusion Matrix:
      0    1
0  202   40
1   32  178

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.83      0.85       242
           1       0.82      0.85      0.83       210

    accuracy                           0.84       452
   macro avg       0.84      0.84      0.84       452
weighted avg       0.84      0.84      0.84       452



#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 3rd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [13]:
# Add anomaly detection code  (Question #E209)
# ----------------------------------
# Construct IsolationForest
iso_forest = IsolationForest(n_estimators = 250)

iso_outliers = iso_forest.fit(X_train, y_train)

# Get labels from classifier to cull outliers
iso_outliers = iso_forest.predict(X_train) == -1

X_iso = X_train[~iso_outliers]
y_iso = y_train[~iso_outliers]


In [21]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E210)
# ----------------------------------
# Define pipeline
'''pipe3 = Pipeline(
    [
        #('selector', FactorAnalysis()),
        #('classifier', VarianceThreshold())
    ]
)

param_grid3 = [
    {
        'selector__n_components': [5, 20, 80, 120, 480],
        'selector__random_state': [17],
        'classifier__threshold': [0, 0.0001, 0.001, 0.01, 0.1]
    }
]'''

pipe3 = Pipeline(
    [
        ('selector', SelectPercentile()),
        ('classifier', RandomForestClassifier())
    ]
)

param_grid3 = [
    {
        'selector__percentile': [1, 5, 10, 50, 75],
        'classifier__max_features' : ['auto', 'sqrt', 'log2'],
        'classifier__max_depth': [None, 55, 30, 10, 1],
        'classifier__n_estimators':[25, 50, 100, 150, 500],
        'classifier__random_state': [17]
    }
]

# Use Grid Search to train Pipeline
CV_rfc = GridSearchCV(pipe3, param_grid = param_grid3, n_jobs = 2, cv = 5)
CV_rfc_model = CV_rfc.fit(X_iso, y_iso)
#print(CV_rfc.cv_results_)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [12 15] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [22]:
# Given an unbiased evaluation  (Question #E211)
# ----------------------------------
## Evaluate best model using test data
# Make prediction using test data
predicted_y = CV_rfc.predict(X_test)

'''# Show parameters of best model
best_params = CV_rfc.best_params_
print('Best parameter:\n',best_params)'''

# Show best estimator
best_estimator = CV_rfc.best_estimator_
print('\nBest estimator:\n',best_estimator) 

# Display confusion matrix
print('\nConfusion Matrix:\n',pd.DataFrame(confusion_matrix(y_test, predicted_y)))

'''#Display accuracy score
print('\nAccuracy Score:\n', pd.DataFrame(accuracy_score(y_test.actual_label.values, predicted_y.predicted_RF.values)))'''

# Create classification report
print('\nClassification Report:\n',classification_report(y_test, predicted_y))



Best estimator:
 Pipeline(steps=[('selector', SelectPercentile(percentile=75)),
                ('classifier',
                 RandomForestClassifier(max_depth=10, n_estimators=500,
                                        random_state=17))])

Confusion Matrix:
      0    1
0  212   30
1   19  191

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.88      0.90       242
           1       0.86      0.91      0.89       210

    accuracy                           0.89       452
   macro avg       0.89      0.89      0.89       452
weighted avg       0.89      0.89      0.89       452



#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## Compare these three pipelines and discuss your findings

## <span style="background: yellow;">Commit your code!</span> 

### Pickle the required pipeline/models for Part III.

In [23]:
# Pickle the best pipeline
joblib.dump(CV_rfc.best_estimator_, 'data/pipeline-v5.pkl')


['data/pipeline-v5.pkl']

You should have made a few commits so far of this project.  
**Definitely make a commit of the notebook now!**  
Comment should be: `Final Project, Checkpoint - Pipelines done`


# Save your notebook!
## Then `File > Close and Halt`